In [143]:
'''Здесь взят отфильтрованный по протоколу TCP в Wireshark трафик, здесь записи взаимодействия ботнета и машины.
Аномальные записи были помечены красным и черным в интерфейсе Wireshark, поэтому мне было просто отличить их от чистых.
Сбор датасета Okey был произведен вручную: вырезались из исходного датасета чистые фрагменты и собирались в единый список,
поэтому по таймингу есть пустые места. Используется информация из графы Info.'''
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd
import numpy as np
import re
import pandas as pd

In [ ]:
df = pd.read_csv('B:\Учёба\диплом\трафик\Okay.csv')

In [ ]:
'''Понадобилось почистить от знаков препинания, чтоб токенизация прошла лучше.'''
def reformat(text):
    return re.sub("[,|<|>|=|\|/|.]",' ',text)
text = [reformat(i) for i in df['Info']]

In [ ]:
# После очистки проведена токенизация с длиной строки в 20, после чего посмотели на более часто встречающиеся слова.
tokenizer = Tokenizer(num_words = 20)
tokenizer.fit_on_texts(text)
# произведена замена слов на индексы и получилось следующее - некоторые записи не были распознаны
sequences = tokenizer.texts_to_sequences(text)
new_sequences = sequences[:len(text)]

In [149]:
# Здесь не критично, что в массиве new_sequences есть пустые записи, 
# их не будем принимать за аномалии, а просто удалим.
empt = [i for i,x in enumerate(new_sequences) if x ==[] ]
new_text = np.array(text)
new_a = np.delete(text, empt)

In [152]:
sequences = tokenizer.texts_to_sequences(new_a)

In [153]:
'''Дальше формируется обучающий датасет, с маленьким коэффициентом аномальности, от 4 до 8'''
x_train = pad_sequences(sequences, maxlen = 15)
df_normal = pd.DataFrame(new_a, columns = ['tokens'])
df_normal['expect'] = [np.sum(i)/len(i) for i in sequences]

In [155]:
'''Датасет с аномальными данными собирался по томуже принципу, что и с чистыми, 
здесь повторяются те же действия, что и выше.'''
df_anom = pd.read_csv('B:\Учёба\диплом\трафик\Anomaly.csv')
text_anom = [reformat(i) for i in df_anom['Info']]

tokenizer.fit_on_texts(text_anom)

sequences_anom = tokenizer.texts_to_sequences(text_anom)

empt_anom = [i for i,x in enumerate(sequences_anom) if x ==[] ]

new_text = np.array(text_anom)
new_anom = np.delete(text_anom, empt_anom)

sequences_anom = tokenizer.texts_to_sequences(new_anom)

df_anomaly = pd.DataFrame(new_anom, columns = ['tokens'])
df_anomaly['expect'] = [np.sum(i)/len(i)+15 for i in sequences_anom]

In [164]:
'''После всех преобразований сформирован единый датасет и добавлено время.'''
new_df = pd.concat([df_normal,df_anomaly])

df_timeseries = df['Time'].iloc[:len(new_df)]

new_df['time'] = df_timeseries

new_df.to_csv('B:\Учёба\диплом\трафик\For_lstm.csv')